In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
import shutil
from pathlib import Path
from sorted_alpha import sorted_alpha
from scipy.spatial import distance
from sklearn.neighbors import NearestNeighbors
from scipy import interpolate
from frame_count import frame_count
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d

In [ ]:
folder = '/Users/andrei-macpro/Documents/Data/pose/play_openpose/1043_play_1'
poses = sorted_alpha(folder)

In [ ]:
frame_no = [int((x.split("."))[0].split("_")[1]) for x in poses]

In [ ]:
def get_skeletons(frame_no, detection):
    """function that reads a detections frame, checks if necks are present, and returns the skeletons"""
    data = pd.read_csv(detection, index_col=0)
    ## first check if it contains the neck 
    temp_data = data
    for i in range(0, len(data.columns),3): # drop skeletons without neck
        if np.isnan(data[data.columns[i]].loc[1]) == True:
            temp_data = temp_data.drop([data.columns[i],data.columns[i+1], data.columns[i+2]], axis=1) ## drop skeletons with missing neck
    data=temp_data
    if len(data.columns) < 4: ## if not enough skeletons with neck present return 0 
        return(None)
    elif frame_no ==1: 
        if len(data.columns)>6:
            track_1 = data[['x_1', 'y_1']].to_numpy()
            track_2 = data[['x_2', 'y_2']].to_numpy()
            track_3 = data[['x_3', 'y_3']].to_numpy()
            return([track_1, track_2, track_3])
        elif len(data.columns)>4:  ### when first starting tracking initialise with either one or two skeletons 
            track_1 = data[['x_1', 'y_1']].to_numpy()
            track_2 = data[['x_2', 'y_2']].to_numpy()
            return([track_1, track_2])
        else: 
            track_1 = data[['x_1', 'y_1']].to_numpy()
            return([track_1])
    else:
        skeletons=[]
        ## remove probs columns
        for col in data.columns:
            if data[col].name[0:3] == 'pro':
                data=data.drop([col], axis=1)
     # return every skeleton now  
        
        for x,y, in zip(data.columns[::2], data.columns[1::2]):
            skeletons.append(data[[x,y]].to_numpy())
        return(skeletons) ## this is a list of skeletons(stacked)
        
    

In [ ]:
array_nn = [np.array([499.2 , 161.28]), np.array([537.6 , 368.64])]
array_fit = [np.array([506.88, 161.28]), np.array([537.6 , 362.88])]

neigh = NearestNeighbors(n_neighbors=1, metric='euclidean', algorithm='brute')
neigh.fit(array_fit)
neighb = neigh.kneighbors(array_nn)
neighb

In [ ]:
neighb[1][0][0]

In [ ]:
track_1, track_2, track_3 = [], [], []
tracks = [track_1, track_2, track_3]
frames = []
for pose, no in zip(poses, frame_no):
    print(no)
    frames.append(frame_no)
    detect = get_skeletons(no, os.path.join(folder, pose))
    if detect == None: continue
    elif no ==1:
        for detection in range(len(detect)):
            tracks[detection].append(detect[detection])
    else:
        neighb = nn(tracks, detect)
        for n in range(len(neighb)):    
            tracks[int(neighb[1][n][0])].append(detect[n])
    

In [ ]:
track_1

In [ ]:
def nn (prev_detections, new_detect):
    """function that does nn search between the new detections and previous detections"""
    neigh = NearestNeighbors(n_neighbors=1, metric='euclidean', algorithm='brute')
    neigh.fit([x[0][1] for x in prev_detections if x!=[]])
    neighbours = neigh.kneighbors([x[1] for x in new_detect], 1, return_distance=True)
    return(neighbours)
    

In [ ]:
tracks[]

In [ ]:
data_1 = pd.read_csv('/Users/andrei-macpro/Documents/Data/pose/play_openpose/1043_play_1/frame_1.csv')
data_2 = pd.read_csv('/Users/andrei-macpro/Documents/Data/pose/play_openpose/1043_play_1/frame_2.csv')

In [ ]:
df_1 = get_skeletons(1,'/Users/andrei-macpro/Documents/Data/pose/play_openpose/1043_play_1/frame_1.csv')
df_2 = get_skeletons(2,'/Users/andrei-macpro/Documents/Data/pose/play_openpose/1043_play_1/frame_2.csv')

In [ ]:
neigh = NearestNeighbors(n_neighbors=1, metric='euclidean')
neigh.fit([x[0][1] for x in tracks if x!=[]])

In [ ]:
neighbours = neigh.kneighbors([x[1] for x in df_2], 1, return_distance=True)
neighbours

In [ ]:
for n in range(len(neighbours)):
    print(neighbours[n])

In [ ]:
skeletons

In [ ]:
x = [1,5]
y = [[[np.nan,np.nan],[10,11],[12,13]], [[15,16], [17,18], [22.23,20.56]]]
f = interp1d(x, y, axis=0)

In [ ]:
y = [[5,9],[10,11],[0,0],[np.nan,10],[10,11],[12,13], [15,16], [17,18], [19,20]]
mi = pd.MultiIndex.from_arrays([[0,0,0,1,1,1,2,2,2], [1,2,3,1,2,3,1,2,3]], names=['frame', 'joint'])
df = pd.DataFrame(y, columns = ['x','y'], index=mi)

In [ ]:
new_array = f([1,2,3,4,5])


In [ ]:
new_array[-1] = y[-1]

In [ ]:
new_array

In [ ]:
y = [[[5,9],[10,11],[15,16]],[[10,11],[12,13], [np.nan, np.nan]], [[17,18], [19,20],[7,8]],
    [[10,11], [0,8], [9,8]], [[20,4], [19,8], [10,5]]]

savgol_filter(np.array(y), 5, 3, axis=0,mode='interp')

In [ ]:
y = [[[5,9],[10,11],[0,0],[np.nan,10]],[[10,11],[12,13], [15,16]], [[17,18], [19,20],[0,5]],
    [[10,11], [0,8], [9,8]], [[20,4], [19,8], [10,5]]]
np.array(y).shape

In [ ]:
y

In [ ]:
x = 'nan'
x!=x